<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Interpreter

## Intro

This pattern is sued for understanding natural language text. It provides a way to evaluate grammar or an expression.

Given a language, define a representation for its grammar along with an interpreter that uses the representation to interpret sentences in the language.

it usually involved implementing an expression interface which tells how to interpret a particular context

We obvs need to know the grammar of the target language.

When implementing the interpreter pattern, we need to define the grammar representation. We do this by using a class for each rule. Using regular expressions, we can specify matching strings and enforce the rules specifically.

The pattern will describe how to define a grammar for regular expressions, represent a particular expression, and determine how to interpret strings with it.

The most common use case of this pattern is a translator. Same with a compiler.

Advantages of the pattern are:
* Easy to implement if you know the grammar rules
* Can easily extend the language with new grammar rules

A key concern is the number of grammar rules can become difficult to maintain if there are a lot of them. It also requires a lot of code and error handling to evaluate. Also, regular expressions (a key part of the pattern) aren't incredibly performant. It might be better to write a generator or use some other way to evaluate language

## Implementation

In [2]:
class InterpreterContext {
    
    public String getBinaryFormat(int num)
    {
        return Integer.toBinaryString(num);
    }
    
     
    public String getHexadecimalFormat(int num)
    {
        return Integer.toHexString(num);
    }
}

In [3]:
interface Expression {
    String interpret(InterpreterContext ic);
}

In [4]:
class IntToBinaryExpression implements Expression {
    private int num;
    
    public IntToBinaryExpression(int num)
    {
        this.num = num;
    }
    
    public String interpret(InterpreterContext ic)
    {
        return ic.getBinaryFormat(num);
    }
}

In [5]:
class IntToHexExpression implements Expression {
    private int num;
    
    public IntToHexExpression(int num)
    {
        this.num = num;
    }
    
    public String interpret(InterpreterContext ic)
    {
        return ic.getHexadecimalFormat(num);
    }
}

In [6]:
InterpreterContext ic = new InterpreterContext();

In [14]:
Expression myExp = new IntToBinaryExpression(12);
Expression myExp2 = new IntToHexExpression(12);

In [17]:
myExp.interpret(ic);

1100

In [16]:
myExp2.interpret(ic);

c

The pattern does not do any parsing. It merely offers a way to provide parsed input to an interpreter in order to return the expression given the set of rules for the grammar in question (this is a whole thing, basically this thing spits out the representation of the thing we want if the input is correct)

## Example

In [ ]:
interface IContext {
    Boolean getResult(String data);
}

In [6]:
interface Expression {
    Boolean interpret(IContext context);
}

In [7]:
class AppContext implements IContext {
    private String input;
    
    public AppContext(String input)
    {
        this.input = input;
    }
    
    public Boolean getResult(String data)
    {
        if (input.contains(data)) {
            return true;
        }
        
        return false;
    }
}

In [16]:
class TerminalExpression implements Expression {
    private String data;
    
    public TerminalExpression(String data)
    {
        this.data = data;
    }
    
    public Boolean interpret(IContext ctx)
    {
        return ctx.getResult(this.data);
    }
}

In [17]:
class OrExpression implements Expression {
    private Expression exp1; // left branch
    private Expression exp2; // right branch
    
    public OrExpression(Expression e1, Expression e2)
    {
        this.exp1 = e1;
        this.exp2 = e2;
    }
    
    public Boolean interpret(IContext ctx)
    {
        return exp1.interpret(ctx) || exp2.interpret(ctx);
    }
}

In [18]:
class AndExpression implements Expression {
    private Expression exp1; // left branch
    private Expression exp2; // right branch
    
    public AndExpression(Expression e1, Expression e2)
    {
        this.exp1 = e1;
        this.exp2 = e2;
    }
    
    public Boolean interpret(IContext ctx)
    {
        return exp1.interpret(ctx) && exp2.interpret(ctx);
    }
}

In [19]:
Expression left = new TerminalExpression("left");

In [20]:
Expression right = new TerminalExpression("right");

In [21]:
Expression leftOrRight = new OrExpression(left, right);

In [22]:
IContext myCtx = new AppContext("left or right");

In [23]:
leftOrRight.interpret(myCtx);

true

There aren't a lot of moving parts with this pattern. But, they are working in concert continuously. The Expressions are ways of representing the grammar. In the above case, it was simple to create the grammar using logical symbols. These are non-terminal as they are decision making.

The non-terminal expression will need to use recursion to evaluate the rule it holds on the 'child' expressions or the terminals (non-terminals can have non-terminals as children, hence the need for recursion).

The Terminal expression will need to return themselves and/or check themselves against the context in question. The context is the driver here. The context determines how the interpretation will work. In the above case, it is simply checking the above string is contained in the one set in the context. This could become much more complex.

To summarise how the context and expressions work together:
* Expressions contain the grammar (the rules for interpreting)
* Terminal expressions are combined to create a 'language'
* Non-terminal expressions are used to check if the language rules (or grammar) are met
* The terminal interpret and non-terminal interpret will work in concert via recursion.
* The context contains __how__ the grammar rules will be applied. 

Best way to think of this is to think of the context as being what it sounds like. Grammar rules mean nothing without a context. The expressions (non-terminal and terminal) make up the grammar, but the __context__ is what allows that grammar to actually be used.